<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [7]:
!pip install catboost

In [8]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 11.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Подготовка

In [9]:
import pandas as pd
import numpy as np
import nltk
import re 
import lightgbm as lgb
import catboost as cb
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import spacy
from sklearn.pipeline import Pipeline

In [10]:
df = pd.read_csv('datasets/toxic_comments.csv')

In [11]:
df

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [13]:
df.duplicated().sum()

0

In [14]:
print(df[df['toxic']==0].shape)
print(df[df['toxic']==1].shape)

(143106, 3)
(16186, 3)


Дальше когда разделять выборку буду, сохраню баланс классов

с данными всё в порядке</font><font color='Blue'><b> ,но присутсвует большой дизбаланс,</b></font><br>будем работать дальше

In [15]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]' , ' ', text) 
    text = text.lower()
    return " ".join(text.split())

Создадим очистку

In [16]:
df['text'] = df['text'].apply(clear_text)

In [17]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [18]:
def lemmatize(text):
    doc = nlp(text)
    doc=" ".join([token.lemma_ for token in doc])
    return "".join(doc)

создадим функцию лематизации

In [19]:
features=df['text']
target=df['toxic']

разделим выборку...

In [20]:
features = features.apply(lemmatize)


</font><font color='Blue'><b>было долго, но результат получен.
    </b></font><br>

In [21]:
features

0         explanation why the edit make under my usernam...
1         d aww he match this background colour I m seem...
2         hey man I m really not try to edit war it s ju...
3         more I can t make any real suggestion on impro...
4         you sir be my hero any chance you remember wha...
                                ...                        
159287    and for the second time of ask when your view ...
159288    you should be ashamed of yourself that be a ho...
159289    spitzer umm there s no actual article for pros...
159290    and it look like it be actually you who put on...
159291    and I really don t think you understand I come...
Name: text, Length: 159292, dtype: object

In [22]:
df['text']

0         explanation why the edits made under my userna...
1         d aww he matches this background colour i m se...
2         hey man i m really not trying to edit war it s...
3         more i can t make any real suggestions on impr...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287    and for the second time of asking when your vi...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm theres no actual article for prost...
159290    and it looks like it was actually you who put ...
159291    and i really don t think you understand i came...
Name: text, Length: 159292, dtype: object

In [23]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dom\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.1, random_state=12345,stratify=target)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size = 0.25, random_state = 12345,stratify=target_test)

In [25]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train) 

In [26]:
features_test = count_tf_idf.transform(features_test)

In [27]:
features_valid = count_tf_idf.transform(features_valid)

In [28]:
features_train.shape

(143362, 142647)

In [29]:
features_test.shape

(11947, 142647)

In [30]:
#features_valid.shape

с признаками всё нормально, работаем дальше

In [31]:
def itog(model, features_train=features_train, target_train= target_train,features_valid=features_valid, target_valid=target_valid): 
    model.fit(features_train,target_train)
    return print(f1_score(target_valid, model.predict(features_valid)))

функция для оценки моделей, взял из прошлого проекта.

## Обучение

Чутка переберём параметры

LogisticRegression

In [32]:
pipe_lr = Pipeline([ ("vect", TfidfVectorizer(stop_words=stopwords) ),           
 ('clf', LogisticRegression(random_state=12345, class_weight = 'balanced',penalty='l2'))],)

In [33]:
pipe_cat = Pipeline([ ("vect", TfidfVectorizer(stop_words=stopwords) ),           
 ('clf', cb.CatBoostClassifier(loss_function='Logloss', eval_metric='F1',random_state=12345))],)

In [34]:
pipe_tree = Pipeline([ ("vect", TfidfVectorizer(stop_words=stopwords) ),           
 ('clf', DecisionTreeClassifier(random_state = 12345))],) 

In [35]:
model = LogisticRegression(random_state=12345, class_weight = 'balanced',penalty='l2')
parameters = [ {'C':[1, 10,100],}]
lr = GridSearchCV(model,  
                           param_grid = parameters,
                           scoring = 'f1',
                           cv =3,
                           verbose=0,
                           n_jobs = -1)
lr.fit(features_train,target_train)


GridSearchCV(cv=3,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=12345),
             n_jobs=-1, param_grid=[{'C': [1, 10, 100]}], scoring='f1')

In [36]:
print(f' качество модели по кросс-валидации равно {lr.best_score_}')

 качество модели по кросс-валидации равно 0.76408279293703


DecisionTreeClassifier

In [37]:
model = DecisionTreeClassifier(random_state = 12345)
tree_para = {'max_depth':[50,100]}
model = GridSearchCV(model, tree_para, cv=3,scoring = 'f1')
#itog(model)
model.fit(features_train,target_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'max_depth': [50, 100]}, scoring='f1')

In [38]:
print(f' качество модели по кросс-валидации равно {model.best_score_}')

 качество модели по кросс-валидации равно 0.7260519940991955


CatBoostClassifier

In [39]:
model = cb.CatBoostClassifier(loss_function='Logloss', eval_metric='F1',random_state=12345)
grid = {'learning_rate': [0.5, 0.1], 'depth': [3,5],'iterations': [100,200]}     
catboost_grid = model.grid_search(grid,features_train,target_train,cv=3, plot=True, verbose =10)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.3577737	test: 0.3785894	best: 0.3785894 (0)	total: 478ms	remaining: 47.3s
1:	learn: 0.5004711	test: 0.5216960	best: 0.5216960 (1)	total: 809ms	remaining: 39.6s
2:	learn: 0.4495226	test: 0.4691423	best: 0.5216960 (1)	total: 1.14s	remaining: 36.9s
3:	learn: 0.4745390	test: 0.4957703	best: 0.5216960 (1)	total: 1.47s	remaining: 35.3s
4:	learn: 0.4962396	test: 0.5175461	best: 0.5216960 (1)	total: 1.8s	remaining: 34.3s
5:	learn: 0.5154561	test: 0.5458602	best: 0.5458602 (5)	total: 2.13s	remaining: 33.4s
6:	learn: 0.5479988	test: 0.5821934	best: 0.5821934 (6)	total: 2.46s	remaining: 32.7s
7:	learn: 0.5302861	test: 0.5630784	best: 0.5821934 (6)	total: 2.79s	remaining: 32s
8:	learn: 0.5603339	test: 0.5867493	best: 0.5867493 (8)	total: 3.11s	remaining: 31.5s
9:	learn: 0.5640531	test: 0.5922584	best: 0.5922584 (9)	total: 3.44s	remaining: 31s
10:	learn: 0.5673563	test: 0.5963086	best: 0.5963086 (10)	total: 3.76s	remaining: 30.4s
11:	learn: 0.5747989	test: 0.6043031	best: 0.6043031 (11)

Ну очееееень долго и результат не впечатлил.

In [40]:
catboost_grid['params']

{'depth': 5, 'iterations': 200, 'learning_rate': 0.5}

In [41]:
df_t = pd.DataFrame(catboost_grid['cv_results'])
print('средняя катбуста f1 = ',df_t['test-F1-mean'].mean())

средняя катбуста f1 =  0.7195683103670371


Итого мой выбор на логическую регрессию.

Тест с лучшими параметрами

In [42]:
print('F1 мера лучшей модели на тестовых данных:', f1_score(target_test, lr.predict(features_test)))

F1 мера лучшей модели на тестовых данных: 0.768588770864947


## Выводы

Проведен анализ моделей и выбрана лучшая, логическая  регрессию с результато на тестовой выборке = 0.7683018867924527
что яв-ся выше 0.75.
Но на мой взгляд больше времени и можно наверное почти каждую модель разогнать перебором гиперпараметров выше 0,75